In [3]:
# 488ccbf29a8c85fd0bb244400e0beb0f25f0eb5ae25caa3c
# target
# feature_trn_paired shape [time,feature]: (3600, ?)
# feature_test_paired shape[time, feature]: (600, ?)
# brain_trn shape [time,feature]:  (3600, XXX)
# brain_test shape [time,feature]:  (600, XXX)

# train_X shape: (4699, 1024)
# train_Y shape: (4699, 292559)
# test_X shape: (1175, 1024)
# test_Y shape: (1175, 292559)
import numpy as np
import pickle
import random
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import fdrcorrection as fdr
# import matplotlib as mpl
# from scipy.io import loadmat
# import cortex
from PIL import Image
# from matplotlib.backends.backend_pdf import PdfPages
from scipy import stats
import h5py

from ridge_alexhuth import bootstrap_ridge

subj = 'sub_FR025' #sub_EN0109
# Generating random data for feature_trn_paired and feature_test_paired
feature_trn = np.random.rand(4699, 1024)
feature_test = np.random.rand(1175, 1024)
brain_trn =  np.random.rand(4699, 292559)
brain_test = np.random.rand(1175, 292559)


n_voxels = brain_trn.shape[1] # ボクセル数
print('n_voxels: ', n_voxels)

# feature_trn = feature[:3600]
# feature_test = feature[7200:]


def make_pair_data(feature, width, delay):
    '''
    ペアデータを作る
    
    <<parameters>>
    feature: ペアデータにするデータ
    width: 時間幅
    delay: 遅延
    
    <<return>>
    paired_data: ペアデータになったもの
    '''
    for i in range(width):
        a = np.roll(feature, delay+i, axis=0)
        if i==0:
            paired_data = a
        else:
            paired_data = np.concatenate((paired_data, a), axis=1)
    return paired_data

width = 4
delay = 3

feature_trn_paired = make_pair_data(feature_trn, width, delay)
feature_test_paired = make_pair_data(feature_test, width, delay)

print('feature_trn_paired shape:', feature_trn_paired.shape) # (データ数、　次元*4)
print('feature_test_paired shape:', feature_test_paired.shape)


print('brain_trn shape: ', brain_trn.shape) # (データ数、脳活動データの次元)
print('brain_test shape: ', brain_test.shape)

# 更快的平替
wt, corr, valphas, bscorrs, valinds = bootstrap_ridge(feature_trn_paired, brain_trn, feature_test_paired, brain_test,
                                                      alphas=alphas,
                                                      nboots=5,
                                                      chunklen=10, nchunks=15, return_wt=False)
print(wt)
print(corr)
print(valphas)
print(bscorrs)
print(valinds)


ModuleNotFoundError: No module named 'ridge_alexhuth'

In [ ]:
import cortex
import nibabel as nib
from nilearn import plotting
def plot_flatmap(pred_acc, volume_size, subj, xfm, cmap="hot", path=""):
    vol_to_ver_map = cortex.get_mapper(subj, xfm, 'line_nearest', recache=True)

    epi_data = np.zeros(volume_size)
    epi_data[vol_to_ver_map.mask] = np.array(pred_acc)#.reshape(-1)
    # epi_data = epi_data.reshape((volume_size[1], volume_size[0], -1), order="F")
    #
    # max_val = epi_data.max()
    # min_val = 0  # This case uses the minimum value of 0, but you can use any values (e.g. the minimum prediction accuracy that reaches the significance level)
    #
    # # Transform the raw EPI coordination to the pycortex coordination
    # rot_data = np.transpose(epi_data, (2, 1, 0))
    # # Show the cortical map
    plt.figure(figsize=(10, 5))
    # plt.imshow(np.array(Image.open(out_ctxmap_file)))
    plt.axis("off")
    plt.tight_layout()
    # Set volume data
    # dv = cortex.Volume(rot_data, subj, xfm, cmap=cmap, vmin=min_val, vmax=max_val)
    # Generate a flattened cortical map of the volume data and save it as a png file
    # ≈ cortex.quickflat.make_figure(dv, with_colorbar=False)
    _ = cortex.quickflat.make_figure(cortex.Volume(epi_data, subj, xfm,cmap=cmap))
    plt.plot()
    plt.show()

    roi_array_volume = cortex.Volume(epi_data, subj, xfm,cmap=cmap)
    affine = nib.load(
        "/Storage2/ying/pyCortexProj/venv/share/pycortex/db/" + subj + "/anatomicals/brainmask.nii.gz").affine
    _ = plotting.plot_img(nib.Nifti1Image(roi_array_volume.data, affine), display_mode="x")

    plt.savefig(path + '_predictions.png')
    plt.show()